In [1]:
# Python 3.11.7

%pip install scipy
%pip install numpy
%pip install matplotlib
%pip install seaborn
%pip install tensorflow
%pip install sweetviz

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sweetviz as sw
import seaborn as sns
sns.set()

In [5]:
# Let's define the URL for the Shakespeare dataset:
path = "https://raw.githubusercontent.com/schutera/tiny_schiller/main/tiny_schiller.txt"

# Let's download the dataset and save it as 'shakespeare.txt':
filepath = tf.keras.utils.get_file("schiller.txt", path)

# Let's open and read the downloaded text file:
with open(filepath) as f:
    schiller_text = f.read()

print(schiller_text[:100])

Friedrich Schiller
Briefe über Don Carlos.
Erster Brief.
Sie sagen mir, lieber Freund, daß Ihnen die


In [7]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(schiller_text)

max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([schiller_text]))-1

69


In [8]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps=100
window_length=n_steps+1
dataset=dataset.repeat().window(window_length,shift=1,drop_remainder=True)

dataset=dataset.flat_map(lambda window: window.batch(window_length))

batch_size=1024
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
dataset=dataset.map(lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch))
dataset=dataset.prefetch(1)

In [10]:
model=keras.models.Sequential([
    keras.layers.GRU(256, return_sequences=True, input_shape=[None, max_id], dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, dropout=0.2),
    keras.layers.Dense(256),
    keras.layers.Dense(128),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))
])

In [12]:
vlr = keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')
history=model.fit(dataset,
                  steps_per_epoch=train_size/batch_size,
                  epochs=20,
                  callbacks=[vlr])

Epoch 1/20
1793/1793 [==============================] - 455s 248ms/step - loss: 1.9203 - lr: 0.0010
Epoch 2/20
1793/1793 [==============================] - 438s 244ms/step - loss: 1.6791 - lr: 9.5000e-04
Epoch 3/20
1793/1793 [==============================] - 434s 242ms/step - loss: 1.6260 - lr: 9.0250e-04
Epoch 4/20
1793/1793 [==============================] - 434s 242ms/step - loss: 1.6000 - lr: 8.5737e-04
Epoch 5/20
1793/1793 [==============================] - 435s 243ms/step - loss: 1.5843 - lr: 8.1451e-04
Epoch 6/20
1793/1793 [==============================] - 436s 243ms/step - loss: 1.5735 - lr: 7.7378e-04
Epoch 7/20
1793/1793 [==============================] - 433s 241ms/step - loss: 1.5657 - lr: 7.3509e-04
Epoch 8/20
1793/1793 [==============================] - 433s 242ms/step - loss: 1.5597 - lr: 6.9834e-04
Epoch 9/20
1793/1793 [==============================] - 432s 241ms/step - loss: 1.5550 - lr: 6.6342e-04
Epoch 10/20
1793/1793 [==============================] - 431s 240ms/

In [14]:
model_name = 'drunkenschiller'
model.save(model_name, save_format='h5')

In [ ]:
def preprocess(texts):
    X=np.array(tokenizer.texts_to_sequences(texts))-1
    return tf.one_hot(X,max_id)

def next_char(text,temperature=1):
    X_new=preprocess([text])
    y_proba=model.predict(X_new)[0,-1:,:]
    rescaled_logits=tf.math.log(y_proba)/temperature
    char_id=tf.random.categorical(rescaled_logits,num_samples=1)+1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text,n_chars=50,temperature=0.1):
    for _ in range(n_chars):
        text+=next_char(text,temperature)
    return text

In [ ]:
print("Some predicted texts for word 'Nachttischlampe' are as follows:\n ")
for i in range(3):
  print(complete_text('Nachttischlampe'))
  print()

Some predicted texts for word 'MNachttischlampe' are as follows:
 
1/1 [==============================] - 0s 18ms/step
Nachttischlampern, die sechs fürsten rasch zu denkst, oder fin ge

1/1 [==============================] - 0s 18ms/step
Nachttischlampenbist werde ich mein elend, wir drohen, die wink i

1/1 [==============================] - 0s 28ms/step
Nachttischlampe, davonten weeder um sie wielig für eine gedechtig

